# Problem

The footwear company Vans was impacted by a decrease of consumer demand caused by COVID-19. Consequently, it had to cancel a large number of finished goods orders placed to its manufacturing vendors.

The supply chain of Vans is structured as follow:
1. Vans places finished goods orders to its manufacturing vendors.
2. The manufacturing vendors place the required raw material orders to their raw material suppliers.

Vans could cancel finished goods orders from its vendors. However, it is liable for all raw material orders placed by the vendors to their respective raw material suppliers.

The new season has started, and consumer demand is back to normal.

Now Vans wants to place new finished goods orders to its vendors for the shoe models of the new season, and whenever possible reuse the raw materials that could not be used due to the cancellations.

Help Vans reduce its raw material liabilities by prioritizing the allocation of new finished goods orders to vendor which have the most raw materials components in stock.

The reduction of raw material stocks is called raw material netting.

# Imports

In [1]:
import sys
sys.stdout.flush()

import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import math
import re

from rapidfuzz.process import extractOne, extract_iter
from rapidfuzz.fuzz import ratio, partial_ratio

import pulp as pl

In [2]:
orders = pd.read_excel(r'~/Input Data.xlsx', sheet_name='Finished Goods Orders')
material_bill = pd.read_excel(r'~/Input Data.xlsx', sheet_name='Bill of Materials')
vendor_positions = pd.read_excel(r'~/Input Data.xlsx', sheet_name='Raw Material Positions')

# Processing

In [3]:
orders.rename(columns={'Product#': 'product_raw',
                       'Order Quantity (pairs of shoes)': 'quantity'},
              inplace=True)

orders.head()

,product_raw,quantity
0,VN0001R1GI6,3000
1,VN0004KIIX6,140
2,VN0004KIIX6,134
3,VN0004LG27K,96
4,VN0004LG27K,96


In [4]:
material_bill.rename(columns={'Product#': 'product_raw',
                              'Raw Material Description': 'material_raw',
                              'Raw Material Usage': 'quantity',
                              'Unit of Measure': 'unit'},
                     inplace=True)

material_bill.head()

,product_raw,material_raw,quantity,unit
0,VN0A2RR12RS,1.4-1.6MM COW SUEDE BLANC DE BLANC,0.7538,SF
1,VN0A2RR12RS,TSK20190252 MATERIAL COLOR:CLASSIC WHITE PRIN...,0.0185,Y
2,VN0A2RR12RS,"35""KS13-75 8MM FLAT COTTON BLACK",1.0000,PAIR
3,VN0A2RR12RS,DSI01-2B 10*5*4.5 TRUE WHITE - MATTE,20.0000,Set
4,VN0A2RR12RS,TSK20181746 MATERIAL COLOR: CLASSIC WHITE P...,0.0618,Y


In [5]:
vendor_positions.rename(columns={'Vendor': 'vendor',
                                 'Country of Origin': 'country',
                                 'Raw Material Description': 'material_raw',
                                 'Raw Material Supplier Name': 'supplier',
                                 'Raw Material Quantity': 'quantity',
                                 'Unit of Measure': 'unit',
                                 'Raw Material Costs': 'cost'},
                        inplace=True)

vendor_positions.head()

,vendor,country,material_raw,supplier,quantity,unit,cost
0,S2V,VIETNAM,"0.45MM 54"" SL132 SUPERLIGHT WITH SELF-ADHESIVE...",Hanyong,4.79220,Yard,7.667520
1,S2V,VIETNAM,1.25MM 1*1.5M INSOLE FLEX PLUS，，GREEN,COSMO,22.22100,SH,42.219900
2,S2V,VIETNAM,"1.25MM MIDSOLE FABRIC 54"" SINGLE-SIDED STRIPE ...",YongXin,11.77500,Yard,27.671250
3,S2V,VIETNAM,"10OZ DRILL CANVAS 58""，BLACK",JengChen,83.42360,Yard,181.029212
4,S2V,VIETNAM,145CM ROCK，，#GRIGIO,PRIMA VISIONE,102.21744,M,899.358101


### Text cleaning with regex and replacement

In [6]:
# Orders

orders.replace(to_replace=[r"\\t|\\n|\\r", "\t|\n|\r"], value=["",""], regex=True, inplace=True)

orders['product'] = orders['product_raw'].str.upper()\
                                         .str.normalize('NFKC')\
                                         .str.replace("'", '')\
                                         .str.replace('"', '')\
                                         .str.replace('#', '')\
                                         .str.replace(':', '')\
                                         .str.replace('=', '')\
                                         .str.replace('，', '')\
                                         .str.replace('+', '')\
                                         .str.replace('/', '')\
                                         .str.replace(' ', '')\
                                         .str.replace(',', '')\
                                         .str.replace('(', '')\
                                         .str.replace(')', '')\
                                         .str.replace("_", '')

# Material Bill

material_bill.replace(to_replace=[r"\\t|\\n|\\r", "\t|\n|\r"], value=["",""], regex=True, inplace=True)

material_bill['product'] = material_bill['product_raw'].str.upper()\
                                                       .str.normalize('NFKC')\
                                                       .str.replace("'", '')\
                                                       .str.replace('"', '')\
                                                       .str.replace('#', '')\
                                                       .str.replace(':', '')\
                                                       .str.replace('=', '')\
                                                       .str.replace('，', '')\
                                                       .str.replace('+', '')\
                                                       .str.replace('/', '')\
                                                       .str.replace(' ', '')\
                                                       .str.replace(',', '')\
                                                       .str.replace('(', '')\
                                                       .str.replace(')', '')\
                                                       .str.replace("_", '')

material_bill['material'] = material_bill['material_raw'].str.upper()\
                                                         .str.normalize('NFKC')\
                                                         .str.replace(r'(?<=\d)"', 'INCH', regex=True)\
                                                         .str.replace(r"(?<=\d)'", 'FEET', regex=True)\
                                                         .str.replace('"','')\
                                                         .str.replace("'", '')\
                                                         .str.replace('#', '')\
                                                         .str.replace(':', '')\
                                                         .str.replace('=', '')\
                                                         .str.replace('，', '')\
                                                         .str.replace('+', '')\
                                                         .str.replace('/', '')\
                                                         .str.replace(r'^(.*?)([^a-zA-Z\ \,]*MM)(.*?)$','\\2\\1\\3', regex=True)\
                                                         .str.replace(r'^(.*?)([^a-zA-Z\ \,]*CM)(.*?)$','\\2\\1\\3', regex=True)\
                                                         .str.replace(r'^(.*?)([^a-zA-Z\ \,]*OZ)(.*?)$','\\2\\1\\3', regex=True)\
                                                         .str.replace(' ', '')\
                                                         .str.replace(r'^(.*?)(\d*INCH)(.*)$', '\\2\\1\\3', regex=True)\
                                                         .str.replace(r'^(.*?)(\d*FEET)(.*)$', '\\2\\1\\3', regex=True)\
                                                         .str.replace(',', '')\
                                                         .str.replace('(', '')\
                                                         .str.replace(')', '')\
                                                         .str.replace("_", '')

# Vendor Position

vendor_positions.replace(to_replace=[r"\\t|\\n|\\r", "\t|\n|\r"], value=["",""], regex=True, inplace=True)

vendor_positions['material'] = vendor_positions['material_raw'].str.upper()\
                                                               .str.normalize('NFKC')\
                                                               .str.replace(r'(?<=\d)"', 'INCH', regex=True)\
                                                               .str.replace(r"(?<=\d)'", 'FEET', regex=True)\
                                                               .str.replace("'", '')\
                                                               .str.replace('"', '')\
                                                               .str.replace('#', '')\
                                                               .str.replace(':', '')\
                                                               .str.replace('=', '')\
                                                               .str.replace('，', '')\
                                                               .str.replace('+', '')\
                                                               .str.replace('/', '')\
                                                               .str.replace(r'^(.*?)([^a-zA-Z\ \,]*MM)(.*?)$','\\2\\1\\3', regex=True)\
                                                               .str.replace(r'^(.*?)([^a-zA-Z\ \,]*CM)(.*?)$','\\2\\1\\3', regex=True)\
                                                               .str.replace(r'^(.*?)([^a-zA-Z\ \,]*OZ)(.*?)$','\\2\\1\\3', regex=True)\
                                                               .str.replace(' ', '')\
                                                               .str.replace(r'^(.*?)(\d*INCH)(.*)$', '\\2\\1\\3', regex=True)\
                                                               .str.replace(r'^(.*?)(\d*FEET)(.*)$', '\\2\\1\\3', regex=True)\
                                                               .str.replace(',', '')\
                                                               .str.replace('(', '')\
                                                               .str.replace(')', '')\
                                                               .str.replace("_", '')

### Aggregating material positions & changing total cost to unit cost

In [7]:
vendor_positions = vendor_positions.groupby(['vendor','material_raw','material']).sum().reset_index()
vendor_positions['cost'] = vendor_positions['cost']/vendor_positions['quantity']
vendor_positions.head()

,vendor,material_raw,material,quantity,cost
0,CLC,1.1-1.3MM MARSHMALLOW 11-4300TPG TITIAN PU COA...,1.1-1.3MMMARSHMALLOW11-4300TPGTITIANPUCOATEDLE...,2160.958214,2.595600
1,CLC,1.2-1.4MM BLACK AURORA SUEDE,1.2-1.4MMBLACKAURORASUEDE,100.292732,1.802159
2,CLC,"1.2-1.4MM BLACK BASIC LEATHER, RP B-100",1.2-1.4MMBLACKBASICLEATHERRPB-100,234.840000,1.854000
3,CLC,1.2-1.4MM CHIPMUNK17-1044TPG NATUREBUCK,1.2-1.4MMCHIPMUNK17-1044TPGNATUREBUCK,4458.870000,2.950000
4,CLC,1.2-1.4MM DRIZZLE 16-4402TPG AURORA SUEDE,1.2-1.4MMDRIZZLE16-4402TPGAURORASUEDE,100.292732,2.000000


### Removing duplicates from material bill & pivoting

In [8]:
material_bill_unique = material_bill.drop_duplicates(['product','material'],keep= 'last')

material_bill_piv = material_bill_unique.pivot(index='product', columns='material', values='quantity')\
                                        .reindex(index=orders['product'])\
                                        .fillna(0)\
                                        .drop_duplicates(keep='last')

In [9]:
material_bill_piv.head()

material,.0.7MMPUMERISKIN,0.15MMNASA-LQHOTMELTFILM15CWELTFILM-CX029PPCLEARRELEASEPAPER,0.2MMNASA-TWELDFILM20CWELTFILM-CX034POLYPROPHYLENE,0.3MMFEELMAXHMMOKKAHOTMELTFILM11-4300TPGWHITE,0.3MMFEELMAXHMMOKKAHOTMELTFILM15-1821TPGPINK,0.3MMFEELMAXHMMOKKAHOTMELTFILM17-0000TPGGRAY,0.3MMFEELMAXHMMOKKAHOTMELTFILM18-5203TPGGRAY,0.3MMFEELMAXHMMOKKAHOTMELTFILM19-0417TPGGREEN,0.3MMFEELMAXHMMOKKAHOTMELTFILM19-0608TPGGRAY,0.3MMFEELMAXHMMOKKAHOTMELTFILM19-4024TPGNAVYBLUE,...,YH-S535-1MESHAUTOWEIGHT180GY-10%BLACK,YH-S535-1MESHAUTOWEIGHT180GY-10%NATURAL,YH-S535-1MESHAUTOWEIGHT180GY-10%WHITE,YH-S535-2MESHAUTOWHITE,YH-S535MESHAUTOWEIGHT180GY-10%11-0507TPXNATRUAL,YH-S535MESHAUTOWEIGHT180GY-10%14-4811TPGGREEN,YH-S535MESHAUTOWEIGHT180GY-10%WHITE,YXV-1667GORESTRAPS19-0417TPGGREEN,YXV-1667GORESTRAPS19-4024TPGNAVYBLUE,ZK-007GTXBLACKTPUCOUNTER
product,,,,,,,,,,,,,,,,,,,,,
VN0A2RR12RS,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
VN0A2RR12RT,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
VN0A346Y2JE,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
VN0A3488XOC,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
VN0A348A2N5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Creating Static Dataframes
### Netted Material

In [10]:
netted_material = pd.merge(orders, material_bill_piv, on='product', how='left')\
                    .dropna(axis=0)\
                    .reset_index(drop=True)

netted_material = netted_material.mul(netted_material['quantity'], axis=0)\
                                 .drop(['product_raw','quantity','product'], axis=1)

In [11]:
netted_material.head()

,.0.7MMPUMERISKIN,0.15MMNASA-LQHOTMELTFILM15CWELTFILM-CX029PPCLEARRELEASEPAPER,0.2MMNASA-TWELDFILM20CWELTFILM-CX034POLYPROPHYLENE,0.3MMFEELMAXHMMOKKAHOTMELTFILM11-4300TPGWHITE,0.3MMFEELMAXHMMOKKAHOTMELTFILM15-1821TPGPINK,0.3MMFEELMAXHMMOKKAHOTMELTFILM17-0000TPGGRAY,0.3MMFEELMAXHMMOKKAHOTMELTFILM18-5203TPGGRAY,0.3MMFEELMAXHMMOKKAHOTMELTFILM19-0417TPGGREEN,0.3MMFEELMAXHMMOKKAHOTMELTFILM19-0608TPGGRAY,0.3MMFEELMAXHMMOKKAHOTMELTFILM19-4024TPGNAVYBLUE,...,YH-S535-1MESHAUTOWEIGHT180GY-10%BLACK,YH-S535-1MESHAUTOWEIGHT180GY-10%NATURAL,YH-S535-1MESHAUTOWEIGHT180GY-10%WHITE,YH-S535-2MESHAUTOWHITE,YH-S535MESHAUTOWEIGHT180GY-10%11-0507TPXNATRUAL,YH-S535MESHAUTOWEIGHT180GY-10%14-4811TPGGREEN,YH-S535MESHAUTOWEIGHT180GY-10%WHITE,YXV-1667GORESTRAPS19-0417TPGGREEN,YXV-1667GORESTRAPS19-4024TPGNAVYBLUE,ZK-007GTXBLACKTPUCOUNTER
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Vendor Material Positions

In [12]:
vendor_material_positions = vendor_positions.groupby(['vendor','material'])['quantity']\
                                            .sum()\
                                            .unstack('material')\
                                            .reset_index()\
                                            .fillna(0)\
                                            .set_index('vendor', drop=True)

In [13]:
vendor_material_positions.head()

material,-1681.2-1.4MM13-4910TPGVIRGINLEATHERAR,0.3MMFEELMAXHMMOKKAHOTMELTFILM19-1627TPGPORTROYALE,0.3MMFEELMAXHMMOKKAHOTMELTFILMWHITE,0.45MMSUPERLIGHTWCEMENTCOATINGGREY,0.45MMSUPERLIGHTWCEMENTCOATINGWHITE,0.4MMC04TKTOEBOX,0.4MMTPUHM1*1.5MUNO9TA2,0.4MMTPUHM1*1.5MUNOWHITE,0.4MMTPUHM1M*1.45MSPORTFLEXE259TA2,0.5MMFEELMAXHMTR-70%UMTHOTMELTFILM12-5202TPGTURTLEDOVE,...,“ULTRADEE”–M30BONDEDPOLYESTERTOPANDBOBBINTHREAD12-1310TPGCOLORAE43A,“ULTRADEE”–M30BONDEDPOLYESTERTOPANDBOBBINTHREAD15-1142TPG12GYS,“ULTRADEE”–M30BONDEDPOLYESTERTOPANDBOBBINTHREAD16-1522TPGCOLORAS11N,“ULTRADEE”–M30BONDEDPOLYESTERTOPANDBOBBINTHREAD16-5106TPGCOLORAS07X,“ULTRADEE”–M30BONDEDPOLYESTERTOPANDBOBBINTHREAD17-0000TPGCOLOR66DYS,“ULTRADEE”–M30BONDEDPOLYESTERTOPANDBOBBINTHREAD18-1033TPGCOLOR658SR,“ULTRADEE”–M30BONDEDPOLYESTERTOPANDBOBBINTHREAD19-0511TPGCOLORAS10D,“ULTRADEE”–M30BONDEDPOLYESTERTOPANDBOBBINTHREADBLACKG9600COLOR,“ULTRADEE”–M30BONDEDPOLYESTERTOPANDBOBBINTHREADNATRUAL36UCQCOLOR,“ULTRADEE”–M30BONDEDPOLYESTERTOPANDBOBBINTHREADTRUEWHITEG9401COLOR
vendor,,,,,,,,,,,,,,,,,,,,,
CLC,0.0,0.000,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.000,0.000,0.000
CLK,0.0,0.000,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.05,84.76,2.27,0.11,0.29,0.87,4.2,1303.526,1289.617,3003.477
DJC,0.0,0.000,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.000,0.000,0.000
DTP,0.0,0.000,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.000,0.000,0.000
FSC,0.0,8.532,8.532,0.0,0.0,0.0,0.0,0.0,0.0,1.867168,...,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.000,0.000,0.000


### Vendor Material Cost

In [14]:
vendor_material_cost = vendor_positions.groupby(['vendor','material'])['cost']\
                                       .sum()\
                                       .unstack('material')\
                                       .reset_index()\
                                       .fillna(0)\
                                       .set_index('vendor', drop=True)

In [15]:
vendor_material_cost.head()

material,-1681.2-1.4MM13-4910TPGVIRGINLEATHERAR,0.3MMFEELMAXHMMOKKAHOTMELTFILM19-1627TPGPORTROYALE,0.3MMFEELMAXHMMOKKAHOTMELTFILMWHITE,0.45MMSUPERLIGHTWCEMENTCOATINGGREY,0.45MMSUPERLIGHTWCEMENTCOATINGWHITE,0.4MMC04TKTOEBOX,0.4MMTPUHM1*1.5MUNO9TA2,0.4MMTPUHM1*1.5MUNOWHITE,0.4MMTPUHM1M*1.45MSPORTFLEXE259TA2,0.5MMFEELMAXHMTR-70%UMTHOTMELTFILM12-5202TPGTURTLEDOVE,...,“ULTRADEE”–M30BONDEDPOLYESTERTOPANDBOBBINTHREAD12-1310TPGCOLORAE43A,“ULTRADEE”–M30BONDEDPOLYESTERTOPANDBOBBINTHREAD15-1142TPG12GYS,“ULTRADEE”–M30BONDEDPOLYESTERTOPANDBOBBINTHREAD16-1522TPGCOLORAS11N,“ULTRADEE”–M30BONDEDPOLYESTERTOPANDBOBBINTHREAD16-5106TPGCOLORAS07X,“ULTRADEE”–M30BONDEDPOLYESTERTOPANDBOBBINTHREAD17-0000TPGCOLOR66DYS,“ULTRADEE”–M30BONDEDPOLYESTERTOPANDBOBBINTHREAD18-1033TPGCOLOR658SR,“ULTRADEE”–M30BONDEDPOLYESTERTOPANDBOBBINTHREAD19-0511TPGCOLORAS10D,“ULTRADEE”–M30BONDEDPOLYESTERTOPANDBOBBINTHREADBLACKG9600COLOR,“ULTRADEE”–M30BONDEDPOLYESTERTOPANDBOBBINTHREADNATRUAL36UCQCOLOR,“ULTRADEE”–M30BONDEDPOLYESTERTOPANDBOBBINTHREADTRUEWHITEG9401COLOR
vendor,,,,,,,,,,,,,,,,,,,,,
CLC,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.00
CLK,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,4.9,4.9,4.9,4.9,4.9,4.9,4.9,4.52,4.9,3.76
DJC,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.00
DTP,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.00
FSC,0.0,9.44,9.44,0.0,0.0,0.0,0.0,0.0,0.0,18.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.00


# Fuzzy String Matching Materials

In [16]:
netted_material_cols = list(netted_material.columns)
vendor_material_positions_cols = list(vendor_material_positions.columns)

netted_material_unique = list(set(netted_material_cols)-set(vendor_material_positions_cols))
vendor_material_positions_unique = list(set(vendor_material_positions_cols)-set(netted_material_cols))

unmatched_materials = netted_material_unique + vendor_material_positions_unique
matched_materials = set(netted_material_cols).intersection(set(vendor_material_positions_cols))

print(f'Unmatched columns: {len(unmatched_materials)}')
print(f'Matched columns: {len(matched_materials)}')

Unmatched columns: 4805
Matched columns: 357


In [17]:
fuzzy_match = {}

for query in netted_material_unique:
    
    match = ''
    
    m1 = extractOne(query, vendor_material_positions_unique, scorer=ratio)
    m2 = extractOne(query, vendor_material_positions_unique, scorer=partial_ratio)

    q1, s1 = m1[0], math.ceil(m1[1])
    q2, s2 = m2[0], math.ceil(m2[1])
        
    criterion = (s1>85) or (s2>90) or ((s1+s2>160) and (q1==q2))
    
    if criterion:
        if q1 == q2:
            match = q1
        elif (100-s2)*2 > (100-s1):
            match = q1
        else:
            match = q2
        fuzzy_match[match] = query
         
old_material_keys = [i for i in fuzzy_match.values()]
new_material_keys = list(fuzzy_match.keys())

### Renaming materials in netted materials to match vendor materials

In [18]:
netted_material.rename(columns=dict(zip(old_material_keys, new_material_keys)), inplace=True)

In [19]:
netted_material_cols = list(netted_material.columns)
vendor_material_positions_cols = list(vendor_material_positions.columns)

netted_material_unique = list(set(netted_material_cols)-set(vendor_material_positions_cols))
vendor_material_positions_unique = list(set(vendor_material_positions_cols)-set(netted_material_cols))

unmatched_materials = netted_material_unique + vendor_material_positions_unique
matched_materials = set(netted_material_cols).intersection(set(vendor_material_positions_cols))

print(f'Unmatched columns: {len(unmatched_materials)}')
print(f'Matched columns: {len(matched_materials)}')

Unmatched columns: 4265
Matched columns: 627


### Selecting only matched materials in material columns

In [20]:
netted_material_mm = netted_material[matched_materials]
vendor_material_positions_mm = vendor_material_positions[matched_materials]
vendor_material_cost_mm = vendor_material_cost[matched_materials]

print(netted_material_mm.shape)
print(vendor_material_positions_mm.shape)
print(vendor_material_cost_mm.shape)

(370, 627)
(13, 627)
(13, 627)


# Creating Static Lists
### Vendors

In [21]:
vendors_list = list(vendor_material_cost_mm.index)

### Materials

In [22]:
materials_list = list(vendor_material_positions_mm.columns)

### Orders

In [23]:
orders_list = list(netted_material_mm.index)

# Linear Optimization Model

### Static Lists
$\text{Vendors} = \text{V}$
- $\text{List of vendors}$
- $\text{Length }= 13$

$\text{Orders} = \text{O}$
- $\text{List of orders}$
- $\text{Length }= 370$

$\text{Materials} = \text{M}$
- $\text{List of materials}$
- $\text{Length }= 627$


### Static Dicts

$\text{Vendor Material Cost} = \text{VMC}_{v,m}$

- $\text{Cost of material }m\text{ for vendor } v$
- $\text{Size } = 13 \times 627 = 8151$

$\text{Vendor Material Positions} = \text{VMP}_{v,m}$

- $\text{Positions of material }m\text{ for vendor } v$
- $\text{Size }= 13 \times 627 = 8151$

$\text{Netted Material} = \text{NM}_{o,m}$

- $\text{Required material }m\text{ for order } o$
- $\text{Size }= 370 \times 627 = 231990$


### Decision Variable

$\text{Decision Variable} =  DV_{o,v}$

- $\text{DV}_{o,v} = \begin{cases} 1 & \text{if order } o \text{ is assigned to vendor } v\\ 0 & \text{if order } o \text{ is not assigned to vendor } v\end{cases}$

### Dependent Variable

$\text{Adjusted Inventory} = \text{AI}_{v,m}$

- $\text{Inventory of material } m \text{ for vendor }v\text{ after netting}$
- $\text{AI}_{v,m} \in \mathbb{R}^+$

### Objective Function

$\min(\sum\limits_{v\in V}\sum\limits_{m \in M}\text{AI}_{v,m}*\text{VMC}_{v,m})$

### Constraints

$\sum\limits_{v \in V} \text{DV}_{o,v} = 1; \space \forall o \in O$

- $\text{Each order }o\text{ must be assigned to exactly 1 vendor } v$

$\text{AC}_{v,m} \geq \text{VMP}_{v,m} - \sum\limits_{o \in O}(\text{DV}_{o,v}*\text{NM}_{o,m});\space \forall v\in V, \forall m\in M$

- $\text{Inventory of material } m \text{ for vendor }v\text{ after netting must be greater or equal to the original inventory minus the netted inventory}$
- $\text{The above constraint is an equality when remaining inventory is positive}$

$\text{AC}_{v,m} \geq 0;\space \forall v\in V, \forall m\in M$

- $\text{Inventory of material } m \text{ for vendor }v\text{ after netting must be greater or equal to }0$
- $\text{This constraint stops inventory from taking on negative values after netting}$


# Linear Optimization Implementation with PuLP


In [24]:
%%capture 
solver_list = pl.listSolvers(onlyAvailable=True)

In [25]:
solver_list

['GUROBI', 'PULP_CBC_CMD', 'PULP_CHOCO_CMD']

### Convert static dataframes into dictionaries

In [26]:
vendor_material_positions_d = {(v,m):vendor_material_positions_mm.loc[v,m]
                                     for v in vendors_list 
                                     for m in materials_list}
vendor_material_cost_d = {(v,m):vendor_material_cost_mm.loc[v,m]
                                for v in vendors_list
                                for m in materials_list}
netted_material_d = {(o,m):netted_material_mm.loc[o,m]
                           for o in orders_list
                           for m in materials_list}

### Decision Variable

In [27]:
orderAlloc = pl.LpVariable.dicts("orderAlloc",
                                 ((o, v)
                                     for o in orders_list
                                     for v in vendors_list),
                                 cat='Binary')

### Dependent Variables

In [28]:
adjInventory = pl.LpVariable.dicts("adjInventory",
                                   ((v, m)
                                       for v in vendors_list
                                       for m in materials_list),
                                   lowBound=0,
                                   cat='Continuous')

for v in vendors_list:
    for m in materials_list:
        adjInventory[v,m].setInitialValue(vendor_material_positions_d[v,m])

### Instantiate Model

In [29]:
nettingModel = pl.LpProblem("nettingModel", pl.LpMinimize)

### Objective Function

In [30]:
nettingModel += pl.lpSum([adjInventory[v,m] * vendor_material_cost_d[v,m]
                         for v in vendors_list
                         for m in materials_list])

### Constraints

In [31]:
for o in orders_list:
    nettingModel += pl.lpSum([orderAlloc[o,v] for v in vendors_list]) == 1

In [32]:
for v in vendors_list:
    for m in materials_list:
        nettingModel += adjInventory[v,m] >= 0
        nettingModel += adjInventory[v,m] >= vendor_material_positions_d[v,m] - pl.lpSum([orderAlloc[o,v] * netted_material_d[o,m] for o in orders_list])

### Solve

In [33]:
result = nettingModel.solve(pl.PULP_CBC_CMD(warmStart=True))

In [34]:
pl.LpStatus[nettingModel.status]

'Optimal'

# Results

In [35]:
diff = (vendor_material_cost_mm * vendor_material_positions_mm).sum().sum() - pl.value(nettingModel.objective)

original_liability = (vendor_material_cost * vendor_material_positions).sum().sum()
netting_liability = original_liability - diff

print(f"\033[1mOrg Liability:\033[0m {original_liability.round(2)}")
print(f"\033[1mNet Liability:\033[0m {netting_liability.round(2)}")
print(f"\033[1mAbs Difference:\033[0m {(netting_liability-original_liability).round(2)}")
print(f"\033[1mPct Difference:\033[0m {(100*(netting_liability-original_liability)/original_liability).round(2)}")

Org Liability: 3611192.76
Net Liability: 3455343.7
Abs Difference: -155849.06
Pct Difference: -4.32


In [36]:
result_array = np.zeros([len(orders_list), len(vendors_list)])

for v in nettingModel.variables():
    if v.varValue != 0 and v.name.startswith('orderAlloc'):

        order_idx = int(v.name.split(',')[0].split('(')[1].replace("'",''))
        vendor_idx = int(vendors_list.index(v.name.split(',')[1].split(')')[0].replace('_','').replace("'",'')))

        result_array[order_idx,vendor_idx] = v.varValue

In [37]:
results_df = pd.DataFrame(result_array, columns=vendors_list, index=orders_list)\
               .apply(np.int64)
results_df.head()

,CLC,CLK,DJC,DTP,FSC,FSK,FSV,ICC,LYV,S2V,SJD,SJV,SVV
0,0,0,0,0,0,0,0,0,0,0,0,1,0
1,0,0,0,0,0,0,0,0,0,0,0,1,0
2,0,0,0,0,0,0,0,0,0,0,0,1,0
3,0,0,0,0,0,0,0,0,0,0,0,1,0
4,0,0,0,0,0,0,0,0,0,0,0,1,0


In [38]:
# number of orders allocated to each vendor
results_df.sum()

CLC      4
CLK     46
DJC     29
DTP     85
FSC      6
FSK     40
FSV     14
ICC    103
LYV     13
S2V      8
SJD      2
SJV     12
SVV      8
dtype: int64

In [39]:
# number of orders allocated
results_df.sum().sum()

370